In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
print ('Files in Drive:')

!ls /content/gdrive/'My Drive'

Files in Drive:
 BLSTMCRF_NER_Kashgari_output  'Colab Notebooks'


In [0]:
import os
import pprint
import tensorflow as tf

In [0]:
!pip install kashgari

    100% |████████████████████████████████| 61kB 2.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/6a/4c/96/8cf7dec33f9285762b1466ed34d06995ca4b5f21ed024376b3
  Stored in directory: /root/.cache/pip/wheels/95/5a/29/efc87975860f072feaeebdf3ca47422fb15f40305306e98fac
  Stored in directory: /root/.cache/pip/wheels/df/49/37/76bfc7da796b610556533adfb69747428bff9f14a155d486a1
  Stored in directory: /root/.cache/pip/wheels/39/2d/97/827015c3d00391db39b4759dbfa44ed3c36c5aa638ef8ff300
  Stored in directory: /root/.cache/pip/wheels/a7/4d/34/c56f4459a4def063bb56537fd0055124641032e060807445d6
  Stored in directory: /root/.cache/pip/wheels/e9/10/fb/53ea3dfe7a1e8d61bbdaae8366d8d153b0eef99970f4972814
  Stored in directory: /root/.cache/pip/wheels/dd/0d/77/877e4ef63c477ca4cc4ff18de060d16b0b6a1e09dbe94338c0
  Stored in directory: /root/.cache/pip/wheels/f4/97/a8/9f76e8ab56068514ead6ebef4a7c3d01ab5350223250d39556
Successfully built keras-bert keras-pos-embd keras-transformer keras-multi-head keras-

In [0]:
from kashgari.utils.logger import init_logger
init_logger()

from kashgari.tasks.seq_labeling import BLSTMCRFModel

Using TensorFlow backend.
paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


----------------------
[2019-04-19 15:41:13,439] INFO  - logging init finished


In [0]:
import pandas as pd
df1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/Collection3.csv')
#df1 = pd.read_csv('drive/Colab Notebooks/data/FactRuEval_mix.csv')

df1.head()

,tags,texts
0,LOC O O O O LOC O LOC O O LOC O O O ORG ORG ...,Россия рассчитывает на конструктивное воздейст...
1,O ORG O O O O O O O O O O O LOC O O ORG O O...,Комиссар СЕ критикует ограничительную политику...
2,O O O O O O O O O O O LOC O O LOC O O ORG O ...,"Пулеметы, автоматы и снайперские винтовки изъя..."
3,O O O O O ORG ORG LOC LOC O O O LOC PER PER O...,4 октября назначены очередные выборы Верховног...
4,ORG ORG ORG ORG O O O LOC ORG ORG ORG ORG ORG...,Следственное управление при прокуратуре требуе...


In [0]:
list_of_tags = []
for i in range(0, len(df1['tags'])):
  list_of_tags.append(df1['tags'][i].split(' '))
  
list_of_texts = []
for i in range(0, len(df1['texts'])):
  list_of_texts.append(df1['texts'][i].split(' '))


In [0]:
for i in range(0, len(list_of_texts)):
  if len(list_of_tags[i])!=len(list_of_texts[i]):
    print(i)

In [0]:
# папка, куда распаковать преодобученную нейросеть BERT
#folder = 'multi_cased_L-12_H-768_A-12'
#download_url = 'https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip'  # ссылка на скачивание модели

#print('Downloading model...')
#zip_path = '{}.zip'.format(folder)
#!test -d $folder || (wget $download_url && unzip $zip_path)

#config_path = folder+'/bert_config.json'
#checkpoint_path = folder+'/bert_model.ckpt'
#vocab_path = folder+'/vocab.txt'

In [0]:
folder = 'rubert_cased_L-12_H-768_A-12_v1'
download_url = '/content/gdrive/My Drive/Colab Notebooks/data/rubert_cased_L-12_H-768_A-12_v1.tar.gz'

print('Unpacking model...')
zip_path = '{}.tar.gz'.format(folder)
!test -d $folder || (tar xvzf '/content/gdrive/My Drive/Colab Notebooks/data/rubert_cased_L-12_H-768_A-12_v1.tar.gz')

config_path = folder+'/bert_config.json'
checkpoint_path = folder+'/bert_model.ckpt'
vocab_path = folder+'/vocab.txt'

Unpacking model...
rubert_cased_L-12_H-768_A-12_v1/
rubert_cased_L-12_H-768_A-12_v1/bert_config.json
rubert_cased_L-12_H-768_A-12_v1/vocab.txt
rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt.data-00000-of-00001
rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt.index
rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt.meta


In [0]:
from kashgari.embeddings import BERTEmbedding
embedding = BERTEmbedding('rubert_cased_L-12_H-768_A-12_v1', 400)

[2019-04-19 15:42:06,707] INFO  - loading bert model from rubert_cased_L-12_H-768_A-12_v1

Instructions for updating:
Colocations handled automatically by placer.
[2019-04-19 15:42:06,750] WARNING - From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-04-19 15:42:06,812] WARNING - From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(list_of_texts, list_of_tags, test_size=0.2, random_state=42) 

#делает как мне надо, оригинальными токенами сразу, длины списка токенов совпадают с длиной списка тегов для каждого файла


In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


#model.compile(loss='binary_crossentropy', optimizer= "adam", metrics=[f1])

In [0]:
from keras.callbacks import EarlyStopping

In [0]:
from keras.callbacks import ModelCheckpoint
# checkpoint
filepath="/content/gdrive/My Drive/Colab Notebooks/output_model_NER kashgari_rubert_train_on_Collection3, factrueval_mix, test on NER_Lenta/checkpoints/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_loss', patience=5)
callbacks_list = [checkpoint, es]
#monitor = [f1]

In [0]:
#from kashgari.tasks.seq_labeling import BLSTMCRFModel

# train model
#model = BLSTMCRFModel()
model = BLSTMCRFModel(embedding)
model.fit(X_train, y_train, batch_size = 50, epochs = 50, verbose=0, x_validate=X_test, y_validate=y_test, validation_steps=100, callbacks=callbacks_list)

# save model
model.save('/content/gdrive/My Drive/Colab Notebooks/output_model_NER kashgari_rubert_train_on_Collection3, factrueval_mix, test on NER_Lenta/')

[2019-04-19 17:42:28,190] DEBUG - bert embedding no need to build token2idx with corpus
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 400)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 400)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 400, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 400, 768)     1536        Input-Segment[0][0]              
_____________________

In [0]:
# train model
model.fit(X_train, y_train, batch_size = 24, epochs = 100, verbose=0, x_validate=X_test, y_validate=y_test, validation_steps=100, callbacks=callbacks_list)

# save model
model.save('/content/gdrive/My Drive/Colab Notebooks/output_model_NER kashgari_rubert_train_on_Collection3, factrueval_mix, test on NER_Lenta/')

In [44]:
# train model
model.fit(X_train, y_train, batch_size = 5, epochs = 10, verbose=0, x_validate=X_test, y_validate=y_test, validation_steps=100, callbacks=callbacks_list)

# save model
model.save('/content/gdrive/My Drive/Colab Notebooks/output_model_NER kashgari_rubert_train_on_Collection3, factrueval_mix, test on NER_Lenta/')

[2019-04-19 19:01:36,193] DEBUG - bert embedding no need to build token2idx with corpus
Epoch 1/10
160/160 [==============================] - 311s 2s/step - loss: 0.0812 - crf_accuracy: 0.9704 - val_loss: 0.0795 - val_crf_accuracy: 0.9727
Epoch 2/10
160/160 [==============================] - 310s 2s/step - loss: 0.0578 - crf_accuracy: 0.9782 - val_loss: 0.0762 - val_crf_accuracy: 0.9737
Epoch 3/10
160/160 [==============================] - 309s 2s/step - loss: 0.0476 - crf_accuracy: 0.9819 - val_loss: 0.0846 - val_crf_accuracy: 0.9712
Epoch 4/10
160/160 [==============================] - 309s 2s/step - loss: 0.0438 - crf_accuracy: 0.9835 - val_loss: 0.0860 - val_crf_accuracy: 0.9727
Epoch 5/10
160/160 [==============================] - 310s 2s/step - loss: 0.0379 - crf_accuracy: 0.9855 - val_loss: 0.0815 - val_crf_accuracy: 0.9746
Epoch 6/10
160/160 [==============================] - 309s 2s/step - loss: 0.0369 - crf_accuracy: 0.9860 - val_loss: 0.0833 - val_crf_accuracy: 0.9745
Epoch 

In [60]:
result = []
for i in range(0, len(X_test)):
  ans = model.predict(X_test[i])
  result.append(ans)
  print('predicted: ', ans, ' \n')
  print('true: ', y_test[i], ' \n')

predicted:  ['', 'PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'ORG', 'O', 'ORG', 'O', 'LOC', 'LOC', '', 'O', 'LOC', 'PER', 'PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOC', 'LOC', 'O', 'LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'LOC', '', 'O', 'O', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'ORG', 'ORG', 'ORG', 'LOC', 'LOC', 'O', 'LOC', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'O', 'O', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'O', 'O', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'ORG', 'LOC', 'O', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'ORG', 'LOC', 'O', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'O', 'ORG', 'LOC', 'O', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'ORG', 'LOC', 'O', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'ORG', 'LOC', 'O', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'ORG', 'LOC', 'O', 'LOC', 'LOC', 'O', 'O', 'PER', 'PER', 'O', 'O', 'O', 'O

In [45]:
model.evaluate(X_test, y_test)

IndexError: ignored

In [0]:
f = open('output.txt', 'w')
f.write('Model evaluation: \n')
f.write(model.evaluate(X_test, y_test))
f.write('\n')
f.close()

# Mesure F1 on FactRuEval

In [0]:
df2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/FactRuEval_mix.csv')

df2.head()

In [0]:
list_of_tags = []
for i in range(0, len(df2['tags'])):
  list_of_tags.append(df2['tags'][i].split(' '))
  
list_of_texts = []
for i in range(0, len(df2['texts'])):
  list_of_texts.append(df2['texts'][i].split(' '))


In [0]:
for i in range(0, len(list_of_texts)):
  if len(list_of_tags[i])!=len(list_of_texts[i]):
    print(i)

In [0]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(list_of_texts, list_of_tags, test_size=0.5, random_state=42) 

In [0]:
model.evaluate(X_test_, y_test_)

# Test on Lenta

In [0]:
output_dir = '/content/gdrive/My Drive/Colab Notebooks/data/'

In [49]:
inp = output_dir+'NER_Lenta.csv'
data = pd.read_csv(inp)
data.head()

,textid,textname,full_text
0,20150806moonside,НАСА показало обратную сторону Луны на фоне Земли,Космическому аппарату НАСА DSCOVR удалось полу...
1,20111205piter2,В центре Петербурга задержаны участники протес...,В центре Санкт-Петербурга 5 декабря состоялся ...
2,20161109oilrise,Цена на нефть полностью отыграла потери и нача...,Цена на нефть Brent полностью отыграла потери ...
3,20121008zhirinovskiy,ЛДПР снимет своего кандидата в пользу брянског...,ЛДПР снимет своего кандидата в губернаторы Бря...
4,20120608platforma,Место проведения светских мероприятий &quot;Ки...,В Сочи пожар уничтожил одну из площадок фестив...


In [50]:
%%time
model_result = []
the_result = []
#end = len(data['full_text'])
end = 100
for i in range(0, end):
    if (i % 100) == 0:
      print(i)
    if type(data['full_text'][i]) != float:
        text = data['full_text'][i]
        textid = data['textid'][i]
        textname = data['textname'][i]
        #res = ner_model([text])
        res = model.predict(text)
        model_result.append(res)
        tags = []
        words = []
        for i in range(0, len(res[1][0])):
          if res[1][0][i]!='O':
            tags.append(res[1][0][i])
            words.append(res[0][0][i])

        the_res = [textid, textname, tags, words]
        the_result.append(the_res)

0
CPU times: user 1min 36s, sys: 13.9 s, total: 1min 50s
Wall time: 1min 6s


In [51]:
final_data = []
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru

    100% |████████████████████████████████| 51kB 2.4MB/s 
    100% |████████████████████████████████| 7.1MB 6.0MB/s 
    100% |████████████████████████████████| 8.0MB 5.3MB/s 


In [52]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[2019-04-19 20:12:21,681] INFO  - Loading dictionaries from /usr/local/lib/python3.6/dist-packages/pymorphy2_dicts/data
[2019-04-19 20:12:21,730] INFO  - format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [53]:
for i in range(0, len(the_result)):
    textid = the_result[i][0]
    textname = the_result[i][1]
    pers = []
    orgs = []
    locs = []
    for j in range(0, len(the_result[i][2])):
        if the_result[i][2][j]=='PER':
            text = the_result[i][3][j]
            pers.append(morph.parse(text)[0].normal_form)
        if the_result[i][2][j]=='ORG':
            text = the_result[i][3][j]
            orgs.append(morph.parse(text)[0].normal_form)
        if the_result[i][2][j]=='LOC':
            text = the_result[i][3][j]
            locs.append(morph.parse(text)[0].normal_form)
    final_data.append([textid, textname, pers, orgs, locs])

dataframe = pd.DataFrame(final_data, columns=['textid', 'textname', 'persons', 'organizations', 'locations'])
dataframe.head()

,textid,textname,persons,organizations,locations
0,20150806moonside,НАСА показало обратную сторону Луны на фоне Земли,[],[],[]
1,20111205piter2,В центре Петербурга задержаны участники протес...,[],[],[]
2,20161109oilrise,Цена на нефть полностью отыграла потери и нача...,[],[],[]
3,20121008zhirinovskiy,ЛДПР снимет своего кандидата в пользу брянског...,[],[],[]
4,20120608platforma,Место проведения светских мероприятий &quot;Ки...,[],[],[]


In [0]:
dataframe.to_csv('output.csv', index = False)

In [0]:
for i in range(0, len(the_result)):
  for j in range(0, len(the_result[i][2])):
    if the_result[i][2][j]!='':
      print(the_result[i][2][j])